<center><h1>Data Processing</h1>
    <h3> Author: Daniella Vo and Rohan Subramanian </h3></center>

<br>

<p><center>This notebook is used to convert data from databases other than GEO to the Hegemon file formats. Adapted from Daniella's microbiome processing script. </center></p>

## Import Statements

In [ ]:
%xmode Verbose
import os
import gzip
import shutil
from IPython.display import display
import pandas as pd
from anndata import AnnData
import scanpy as sc 
import operator
import subprocess

print_progress = False
save_extra_files = False

## Parameters

In [ ]:
metadata_file = ''
expression_file = ''
organism = '' # Hs (human) or Mm (mouse)
mapping_file = '' # Genome annotation file with Ensembl ID and gene name

if organism == "Hs":
    mapping_file = "/booleanfs2/sahoo/Data/SeqData/genome/Homo_sapiens.GRCh38.99.chr_patch_hapl_scaff.txt"
if organism = "Mm":
    mapping_file = "/booleanfs2/sahoo/Data/SeqData/genome/Mus_musculus.GRCm38.94.txt"
if organism = "Rn":
    mapping_file = "Rattus_norvegicus.Rnor_5.0.79.txt"

# We have more species' genome annotations in /SeqData/genome/     
    
if metadata_file == '' or expression_file == '' or mapping_file == '':
    raise TypeError('Check input file names')
    
dataset_name = os.getcwd().split('/')[-1].replace('_','-')
print(dataset_name)

## Expression File

In [ ]:
expr = pd.read_csv(expression_file,sep='\t',skiprows=1)
expr = expr.rename(columns={'Gene':'ProbeID'}) # Check what the name is currently, change it
probe_id = expr['ProbeID']
col_names = expr.columns
del expr['ProbeID']

# Normalize Data
adata = AnnData(expr.T)
sc.pp.normalize_total(adata, target_sum=1e6)
sc.pp.log1p(adata, base=2)

norm_expr = pd.DataFrame(adata.X).T

norm_expr.insert(0, 'ProbeID', probe_id)
norm_expr.columns = col_names

# Get gene names from Ensembl IDs
mapping = pd.DataFrame(pd.read_csv(mapping_file, sep = '\t', header=None)) 
mapping.columns = ["ProbeID", "Name"]
norm_expr = mapping.merge(norm_expr,on='ProbeID',how='right')

# Save to file
norm_expr.to_csv(dataset_name+'-expr.txt',sep='\t',index=False)

norm_expr

## Generate survival/IH file

In [ ]:
survival = pd.read_csv(metadata_file,sep='\t')
survival = survival.rename(columns={'sample_name':'ArrayId'})
survival.columns = ['c ' + x if x != 'ArrayId' else x for x in survival.columns ]
survival.insert(1,'time','')
survival.insert(2,'status','')
survival = survival[survival['ArrayId'].isin(expr.columns)]

survival.to_csv(dataset_name+'-survival.txt',sep='\t',index=False)
survival

In [ ]:
ih = pd.DataFrame(survival['ArrayId'])
ih['ArrayHeader'] = ih['ArrayId']
ih['ClinicalHeader'] = ih['ArrayId']

ih.to_csv(dataset_name+'-ih.txt',sep='\t',index=False)
ih

## Make idx file

In [ ]:
expr_file_name = dataset_name+'-expr.txt'
ptr = []
ids = []
name = []
desc = []
pos = 0
print('- Starting idx file generationn')
with open(expr_file_name, 'rb') as f:
    print('- Reading bytes from', expr_file_name)
    for line in f:
        if pos == 0:
            pos += len(line)
        else:
            ptr.append(pos)
            pos += len(line)
            split = line.decode("utf-8").split('\t')
            ids.append(split[0])
            name.append(split[1].split(':')[0])
            desc.append(':'.join(split[1].split(':')[1:]))
    f.close()

with open(dataset_name+'-idx.txt', 'w') as f:
    print('- Writing idx file')
    f.write('ProbeID\tPtr\tName\tDescription\n')
    for i in range(len(ids)):
        f.write('{}\t{}\t{}\t{}\n'.format(ids[i], ptr[i], name[i], desc[i]))
    f.close()
print('- Completed')

## Generate 4 other Hegemon files: bv, info, vinfo, thr

In [ ]:
directory = dataset_name.replace('-','_')

def run_cmd(cmd: str, stderr=subprocess.STDOUT) -> None:
    """Run a command in terminal

    Args:
        cmd (str): command to run in terminal
        stderr (subprocess, optional): Where the error has to go. Defaults to subprocess.STDOUT.

    Raises:
        e: Excetion of the CalledProcessError
    """
    out = None
    try:
        out = subprocess.check_output(
            [cmd],
            shell=True,
            stderr=stderr,
            universal_newlines=True,
        )
    except subprocess.CalledProcessError as e:
        print(f'ERROR {e.returncode}: {cmd}\n\t{e.output}',
              flush=True, file=sys.stderr)
        raise e
    print(out)

cmd = f'bash /booleanfs2/sahoo/Data/BooleanLab/Microbiome/gse_processing {expr_file_name}'
run_cmd(cmd)